In [3]:
import torch
from torch.autograd import Variable
from torch.autograd import Function
from torchvision import models
from torchvision import utils
import cv2
import sys
import numpy as np
import argparse

In [1]:
my_vgg = models.vgg19(pretrained=True)
my_vgg

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (17): ReLU(inplace)

In [149]:
from torch.nn import Conv2d

w = list(my_vgg.features[34].parameters())
print(w[0].shape) # filters
print(w[1].shape) # biases
#print(w[2].shape)

torch.Size([512, 512, 3, 3])
torch.Size([512])


In [150]:
from gradcam import preprocess_image 

image_path = 'examples/both.png'

img = cv2.imread(image_path, 1)
img = np.float32(cv2.resize(img, (224, 224))) / 255
input = preprocess_image(img)

out = my_vgg.features(input)
out.shape

out[0,511,:,:]


tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 3.0775, 7.6843, 2.5410, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000]],
       grad_fn=<SliceBackward>)

In [151]:
# I used this code to find out what the indices are of the last featuremap.
# apparently, when the featuremap is reshaped, the last featuremap are the last 49 elements of the reshaped vector
myfeat = torch.Tensor(out)
myfeat[0,:,:,:] = 0
myfeat[0,511,:,:] = 1
myfeat

x = myfeat.view(myfeat.size(0), -1)
x[0,0] = 5
x[0,25088-7*7-2:] #


tensor([0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       grad_fn=<SliceBackward>)

In [155]:
# here I update the fully connected weights so that 
# for the first 24 pixels the weight is always 1 (of the last featuremap), no matter which output dim
# for the last 24 pixels the weight is always -1 (of the last featuremap), no matter which output dim
# for the middle pixel the weight is 0
# e.g. if the last featuremap is a constant, the contribution to the output of the first fc layer is always 0

with torch.no_grad():
    par = list(my_vgg.classifier[0].parameters())
    par[0].shape # lineair transform part
    par[1].shape # bias part
    
    #par[0].requires_grad_(False)
    
    temp = torch.Tensor(par[0])
    temp.shape
    # 49 / 2 = 24.5
    temp[:,25088-7*7:25088-7*7+24] = -1
    koe = temp[:,25088-7*7:25088-7*7+24]
    print(koe.shape)
    
    temp[:,25088-7*7+25:] = 1
    koe2 = temp[:,25088-7*7+25:]
    print(koe2.shape)
    #temp[:,25088-7*7:] = -1
    
    koe3 = temp[:,25088-7*7:]
    temp[:,25088-7*7+24:25088-7*7+25] = 0
    for i in range(0, koe3.shape[1]):
        print(i)
        print(koe3[1,i])
        
    par[0].copy_(temp)
    
    temp_new = par[0]
    temp_new[:,25088-7*7:]



torch.Size([4096, 24])
torch.Size([4096, 24])
0
tensor(-1., requires_grad=True)
1
tensor(-1., requires_grad=True)
2
tensor(-1., requires_grad=True)
3
tensor(-1., requires_grad=True)
4
tensor(-1., requires_grad=True)
5
tensor(-1., requires_grad=True)
6
tensor(-1., requires_grad=True)
7
tensor(-1., requires_grad=True)
8
tensor(-1., requires_grad=True)
9
tensor(-1., requires_grad=True)
10
tensor(-1., requires_grad=True)
11
tensor(-1., requires_grad=True)
12
tensor(-1., requires_grad=True)
13
tensor(-1., requires_grad=True)
14
tensor(-1., requires_grad=True)
15
tensor(-1., requires_grad=True)
16
tensor(-1., requires_grad=True)
17
tensor(-1., requires_grad=True)
18
tensor(-1., requires_grad=True)
19
tensor(-1., requires_grad=True)
20
tensor(-1., requires_grad=True)
21
tensor(-1., requires_grad=True)
22
tensor(-1., requires_grad=True)
23
tensor(-1., requires_grad=True)
24
tensor(0., requires_grad=True)
25
tensor(1., requires_grad=True)
26
tensor(1., requires_grad=True)
27
tensor(1., requires

In [156]:
with torch.no_grad():
    
    my_conv = list(my_vgg.features[34].parameters())
    print(my_conv[0].shape) # out channels, in channels, width height
    print(my_conv[1].shape) # out channels (bias)
    
    my_filters = my_conv[0]
    my_filters[511,:,:,:] = 0 
    
    my_conv[0].copy_(my_filters)
    
    my_bias = my_conv[1]
    my_bias[511] = 1
    my_conv[1].copy_(my_bias)
    
    print(' ')
    


torch.Size([512, 512, 3, 3])
torch.Size([512])
 


In [170]:
image_path = 'examples/both.png'

img = cv2.imread(image_path, 1)
img = np.float32(cv2.resize(img, (224, 224))) / 255
input = preprocess_image(img)

out = my_vgg.features(input)
out.shape

# because we have changed the filter, the last featuremap is now always constant 1
out[0,511,:,:]

print(out.shape)
out.dtype


torch.Size([1, 512, 7, 7])


torch.float32

In [171]:
out_new = torch.zeros([1, 512, 7, 7], dtype=torch.float32)
print(out_new.shape)

out_new.requires_grad_(True)
out_reshaped = out_new.view(out.size(0), -1)

print(out_reshaped.shape)
print('koe')

lin1 = my_vgg.classifier[0]

out2 = lin1(out_reshaped)
out2.shape

temp = torch.zeros((1,4096))

#temp = torch.tensor((), dtype=torch.float64)
#temp = temp.new_zeros([1,4096])
temp[0] = 1
print(temp.shape)

#out2
out2.backward(temp)

torch.Size([1, 512, 7, 7])
torch.Size([1, 25088])
koe
torch.Size([1, 4096])


In [178]:
out_new.grad[0,508,:,:]

tensor([[-0.2541, -2.2751, -1.6149, -1.4170, -1.7010, -1.8688, -0.3981],
        [-0.9537, -1.9013, -1.7948, -2.5190, -2.2410, -1.8787, -1.4021],
        [-1.4897, -2.7069, -2.5036, -2.4881, -2.2670, -1.9620, -1.5914],
        [-1.8933, -2.8844, -3.2446, -2.5657, -2.9462, -2.7795, -1.5191],
        [-1.8705, -2.5266, -2.6967, -2.6395, -2.8664, -2.5481, -1.9591],
        [-1.8961, -2.8430, -2.8746, -2.9298, -2.3603, -2.9336, -2.4472],
        [-0.9034, -1.1682, -1.3405, -1.7849, -1.5787, -1.2692, -0.9564]])

In [3]:
my_vgg.parameters

<bound method Module.parameters of VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padd

In [2]:
with torch.no_grad():

    new_convlayer = torch.nn.Conv2d(512, 513, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))

    my_param = list(new_convlayer.parameters())

    kernel = my_param[0] # kernel
    bias = my_param[1] # bias
    print(kernel.shape)
    print(bias.shape)

    my_conv = list(my_vgg.features[34].parameters())
    print(my_conv[0].shape) # out channels, in channels, width height
    print(my_conv[1].shape) # out channels (bias)

    vgg_filters = my_conv[0]
    vgg_bias = my_conv[1]

    kernel[0:512,:,:,:] = vgg_filters
    bias[0:512] = vgg_bias

    kernel[512:513,:,:,:] = 0
    bias[512:513] = 1

    my_vgg.features[34] = new_convlayer


# 


torch.Size([513, 512, 3, 3])
torch.Size([513])
torch.Size([512, 512, 3, 3])
torch.Size([512])


In [38]:
with torch.no_grad():

    new_lin = torch.nn.Linear(513 * 7 * 7, 4096)

    param_lin = list(my_vgg.classifier[0].parameters())
    old_A = param_lin[0]
    old_b = param_lin[1]

    new_param = list(new_lin.parameters())
    new_A = new_param[0]
    new_b = new_param[1]

    num = 513 * 7 * 7

    new_A[:,0:512*7*7] = old_A
    new_b.copy_(old_b)

    new_A[:,num-7*7:num-7*7+24] = -1
    #koe = temp[:,num-7*7:num-7*7+24]
    #print(koe.shape)

    new_A[:,num-7*7+25:] = 1
    #koe2 = temp[:,25088-7*7+25:]
    #print(koe2.shape)

    #koe3 = temp[:,25088-7*7:]
    new_A[:,num-7*7+24:num-7*7+25] = 0
    
    my_vgg.classifier[0] = new_lin

RuntimeError: The expanded size of the tensor (25088) must match the existing size (25137) at non-singleton dimension 1.  Target sizes: [4096, 25088].  Tensor sizes: [4096, 25137]

In [7]:
my_vgg

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (17): ReLU(inplace)

In [4]:
def preprocess_image(img):
    means = [0.485, 0.456, 0.406]
    stds = [0.229, 0.224, 0.225]

    preprocessed_img = img.copy()[:, :, ::-1]
    for i in range(3):
        preprocessed_img[:, :, i] = preprocessed_img[:, :, i] - means[i]
        preprocessed_img[:, :, i] = preprocessed_img[:, :, i] / stds[i]
    preprocessed_img = \
        np.ascontiguousarray(np.transpose(preprocessed_img, (2, 0, 1)))
    preprocessed_img = torch.from_numpy(preprocessed_img)
    preprocessed_img.unsqueeze_(0)
    input = Variable(preprocessed_img, requires_grad=True)
    return input

In [41]:
def update_vgg(my_vgg):
    # add extra featuremap, this is the convlayer
    with torch.no_grad():
        new_convlayer = torch.nn.Conv2d(512, 513, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))

        my_param = list(new_convlayer.parameters())

        kernel = my_param[0]  # kernel
        bias = my_param[1]  # bias
        print(kernel.shape)
        print(bias.shape)

        my_conv = list(my_vgg.features[34].parameters())
        print(my_conv[0].shape)  # out channels, in channels, width height
        print(my_conv[1].shape)  # out channels (bias)

        vgg_filters = my_conv[0]
        vgg_bias = my_conv[1]

        kernel[0:512, :, :, :] = vgg_filters
        bias[0:512] = vgg_bias

        kernel[512:513, :, :, :] = 0
        bias[512:513] = 10

        my_vgg.features[34] = new_convlayer

    # add new lineair layer
    with torch.no_grad():
        new_lin = torch.nn.Linear(513 * 7 * 7, 4096)

        param_lin = list(my_vgg.classifier[0].parameters())
        old_A = param_lin[0]
        old_b = param_lin[1]

        new_param = list(new_lin.parameters())
        new_A = new_param[0]
        new_b = new_param[1]

        print(old_A.shape)
        print(new_A.shape)

        num = 513 * 7 * 7

        new_A[:, :] = 10
        new_A[:, 0: 512 * 7 * 7] = old_A

        # 7x7 komen er binnen met een waarde van 1000, en worden daarna nog eens met 1000 vermenigvuldigt

        #new_b[:] = old_b[:] - 7*7*1000*1000
        new_b.copy_(old_b - 7*7*10*10)

        # new_A[:, num - 7 * 7:num - 7 * 7 + 24] = -1
        # koe = temp[:,num-7*7:num-7*7+24]
        # print(koe.shape)

        # new_A[:, num - 7 * 7 + 25:] = 1
        # koe2 = temp[:,25088-7*7+25:]
        # print(koe2.shape)

        # koe3 = temp[:,25088-7*7:]
        # new_A[:, num - 7 * 7 + 24:num - 7 * 7 + 25] = 0

        print(new_A[0, 512 * 7 * 7:])

        my_vgg.classifier[0] = new_lin

img = cv2.imread(image_path, 1)
img = np.float32(cv2.resize(img, (224, 224))) / 255
input = preprocess_image(img)

out = my_vgg(input)
#out.shape

#out[0,512,:,:]
out.shape

torch.Size([1, 1000])

In [6]:
image_path = 'both.png'

#my_vgg

In [42]:

import torch
from torch.autograd import Variable
from torch.autograd import Function
from torchvision import models
from torchvision import utils
import cv2
import sys
import numpy as np
import argparse

#del my_vgg
my_vgg = models.vgg19(pretrained=True)

for mod in my_vgg.features:
    mod.eval()
for mod in my_vgg.classifier:
    mod.eval()
    

img = cv2.imread(image_path, 1)
img = np.float32(cv2.resize(img, (224, 224))) / 255
input = preprocess_image(img)
output_before = my_vgg.features(input)

lin1_bf = my_vgg.classifier[0]
lin1_bf_out = lin1_bf(output_before.view(1, -1))

output_final = my_vgg(input)

update_vgg(my_vgg)

for mod in my_vgg.features:
    mod.eval()
for mod in my_vgg.classifier:
    mod.eval()
    

img = cv2.imread(image_path, 1)
img = np.float32(cv2.resize(img, (224, 224))) / 255
input_new = preprocess_image(img)

output_after = my_vgg.features(input_new)

lin1_af = my_vgg.classifier[0]
lin1_af_out = lin1_af(output_after.view(1, -1))

output_after2 = output_after[:,0:512,:,:]
print(output_after2.shape)
print(output_before.shape)

print(torch.sum(torch.abs(lin1_bf_out-lin1_af_out)))
# after linear layer 
lin1_bf_out-lin1_af_out

output_final_after = my_vgg(input_new)

print(output_final_after - output_final)

print(torch.sum(torch.abs(output_final-output_final_after)))


torch.Size([513, 512, 3, 3])
torch.Size([513])
torch.Size([512, 512, 3, 3])
torch.Size([512])
torch.Size([4096, 25088])
torch.Size([4096, 25137])
tensor([10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10.,
        10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10.,
        10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10.,
        10., 10., 10., 10., 10., 10., 10.], requires_grad=True)
torch.Size([1, 512, 7, 7])
torch.Size([1, 512, 7, 7])
tensor(0.8669, grad_fn=<SumBackward0>)
tensor([[-1.0538e-04, -2.7180e-05, -7.1287e-05, -1.2827e-04, -4.8637e-05,
         -1.1110e-04, -1.5664e-04, -2.9802e-06, -6.9618e-05, -5.4359e-05,
         -6.9022e-05, -7.1526e-05, -4.9114e-05, -1.2136e-04, -5.4836e-05,
         -5.4240e-05, -1.0204e-04, -3.1471e-05, -2.3365e-05, -4.8876e-05,
         -3.1948e-05, -2.6226e-05, -3.0041e-05,  1.3351e-05, -4.1544e-05,
         -2.8133e-05, -7.5340e-05, -7.7486e-05, -1.2994e-05, -7.4625e-05,
         -1.3804e-0

In [16]:
xafter = output_after.view(1, -1)
xafter_np = xafter.data.cpu().numpy()[0]

xbefore = output_before.view(1, -1)
xbefore_np = xbefore.data.cpu().numpy()[0]

N = len(xafter_np)

np.sum(np.abs(xafter_np[0:N-7*7]-xbefore_np))

param_lin = list(my_vgg.classifier[0].parameters())
Anew, bnew = param_lin

result = torch.mm(Anew,torch.t(xafter))
result.shape

Anew[:,25088:]

tensor([[1000., 1000., 1000.,  ..., 1000., 1000., 1000.],
        [1000., 1000., 1000.,  ..., 1000., 1000., 1000.],
        [1000., 1000., 1000.,  ..., 1000., 1000., 1000.],
        ...,
        [1000., 1000., 1000.,  ..., 1000., 1000., 1000.],
        [1000., 1000., 1000.,  ..., 1000., 1000., 1000.],
        [1000., 1000., 1000.,  ..., 1000., 1000., 1000.]],
       grad_fn=<SliceBackward>)

In [11]:
res2 = Anew*(xafter)
res2
res2

tensor([[      0.,       0.,       0.,  ..., 1000000., 1000000., 1000000.],
        [      0.,       0.,       0.,  ..., 1000000., 1000000., 1000000.],
        [      0.,       0.,       0.,  ..., 1000000., 1000000., 1000000.],
        ...,
        [     -0.,       0.,       0.,  ..., 1000000., 1000000., 1000000.],
        [      0.,       0.,      -0.,  ..., 1000000., 1000000., 1000000.],
        [     -0.,       0.,       0.,  ..., 1000000., 1000000., 1000000.]],
       grad_fn=<MulBackward0>)

In [12]:
vgg_original = models.vgg19(pretrained=True)

param_lin_old = list(vgg_original.classifier[0].parameters())
Aold, bold = param_lin_old



In [39]:
torch.sum(torch.abs(Aold[:,0:25088] - Anew[:,0:25088]))

resold = Aold*(xbefore)
torch.sum(torch.abs(resold[:,0:25088] - res2[:,0:25088]))

resultold = torch.mm(Aold, torch.t(xbefore))
x = (resultold-result)
xnp = x.data.cpu().numpy()

corrected = result - 1000*1000*7*7
resultold.type()

'torch.FloatTensor'

In [23]:
torch.sum(torch.abs(bold - bnew))/4096


tensor(48999944., grad_fn=<DivBackward0>)

In [20]:
lin1_af_out - lin1_bf_out

tensor([[-1.4888, -2.1575, -1.1421,  ..., -1.4386, -1.4505,  0.6909]],
       grad_fn=<SubBackward0>)

In [16]:
for mod in my_vgg.features:
    mod.eval()
for mod in my_vgg.classifier:
    mod.eval()
    
y = my_vgg(input_new)
y2 = my_vgg(input_new)

y-y2

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.

In [38]:
import numpy as np

x = output_after.view(1, -1)
x.shape

num = x.data.cpu().numpy()[0]
(np.unique(num)

SyntaxError: unexpected EOF while parsing (<ipython-input-38-6527471edd34>, line 7)

In [35]:
koe = output_after.view(1, -1)
print(koe[-7*7:].shape)
x = koe.data.cpu().numpy()[0]
for num in x[-7*7:]:
    print(num)

torch.Size([1, 25137])
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0


het werkt, want de alpha is heel groot als je kijkt naar de weights in the gradcam. alleen het probleem is dat mijn featuremap op 0 stond, en dus alsnog geen bijdrage kon leveren aan de gradcam. daarom heb ik nu de featuremap op 1000 gezet, en de volgende lineaire layer ook op 1000. hierdoor komt er als het goed is een contributie bij elke feature van precies 7*7*1000*1000. dit trek ik er nu dus van af via de bias, om hier weer hiervoor te corrigeren zodat de uiteindelijke output constant blijft. maar het lijkt er op alsof er nu iets mis gaat. hiervoor werkte het al prima ook in de gradcam. 